# Gathering Transfer Rumors

In the following section, we will build a webscrapper, which gathers all the rumors of which club is interessted in a player which eventually left his club to join a new club. The data will be gathered from TransferMarket.

In [1]:
import pandas as pd
import numpy as np
import re
import time
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.parse import urljoin, urlparse
import hashlib
from functools import lru_cache
import random
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
df=pd.read_csv(r'..\..\DataSources\Processed\Cleaned_Final_Stats_test.csv')
df.head()

,Player,Nation,Squad,Pos,Age,League,MP,Starts,Min,90s,...,AvgDist,Limited_Time,Age_Num,Transfer_Window,Club_Left,League_Left,Club_Joined,League_Joined,Transfer_Fee,Performance_Year
0,Aaron Connolly,IRL,Brighton,FW,Early Twenties,Premier League,10.5,5.0,473.5,5.25,...,0.0,0.5,20.5,2022,Brighton & Hove Albion,Premier League,Venezia FC,Other Leagues,0.0,2021.0
1,Aaron Connolly,IRL,Brighton,FW,Early Twenties,Premier League,10.5,5.0,473.5,5.25,...,0.0,0.5,20.5,2022,Brighton & Hove Albion,Premier League,Hull City,Other Leagues,0.0,2021.0
2,Aaron Hickey,GBR,Bologna,DF,Teenager,Serie A,23.5,22.0,1789.0,19.85,...,0.0,0.5,18.5,2022,FC Bologna,Serie A,FC Brentford,Premier League,14250000.0,2021.0
3,Aaron Leya Iseka,BEL,Toulouse,FW,Early Twenties,Ligue 1,25.0,10.5,1110.5,12.30,...,0.0,0.0,20.5,2020,FC Toulouse,Ligue 2,FC Metz,Ligue 1,0.0,2019.0
4,Aaron Mooy,AUT,Brighton,MF,Late Twenties,Premier League,31.0,25.0,2085.0,23.20,...,0.0,0.0,28.0,2020,Brighton & Hove Albion,Premier League,Shanghai Port,Other Leagues,4500000.0,2019.0


## Getting the Player-ID on Transfermarket of Player

Here we have set up a webscrapper which can gather player-ids on Transfermarket based on URL or player name.

In [91]:
# 1) Build a robust session (reuse yours or replace it)
def build_session() -> requests.Session:
    s = requests.Session()
    # Retries on 429/5xx and some transient network errors
    retry = Retry(
        total=6,
        connect=3,
        read=3,
        status=6,
        backoff_factor=0.6,                 # 0.6, 1.2, 2.4, ...
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"]), # only GET in our scraper
        raise_on_status=False,
        respect_retry_after_header=True,
    )
    adapter = HTTPAdapter(max_retries=retry, pool_connections=50, pool_maxsize=50)
    s.mount("https://", adapter)
    s.mount("http://", adapter)

    s.headers.update({
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                       "(KHTML, like Gecko) Chrome/124.0 Safari/537.36"),
        "Accept-Language": "de-DE,de;q=0.9,en;q=0.8",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Connection": "keep-alive",
    })
    return s

# Use this once at startup
SESSION = build_session()

# 2) A safe fetch wrapper used by ALL your scraping functions
def fetch(
    url: str,
    *,
    referer: str | None = None,
    connect_timeout: float = 5.0,
    read_timeout: float = 25.0,
    tries: int = 3,
    backoff: float = 1.4,
) -> requests.Response:
    """
    GET with extra retries for ReadTimeouts/ConnectTimeouts beyond urllib3 Retry.
    Raises the last exception if all tries fail.
    """
    last_err = None
    headers = {}
    if referer:
        headers["Referer"] = referer

    for attempt in range(1, tries + 1):
        try:
            resp = SESSION.get(url, headers=headers, timeout=(connect_timeout, read_timeout), allow_redirects=True)
            # Retry on 429/5xx here too (even though adapter retries)
            if resp.status_code in (429, 500, 502, 503, 504):
                last_err = requests.HTTPError(f"HTTP {resp.status_code} for {url}")
                # backoff then retry
                sleep = (backoff ** (attempt - 1)) + random.uniform(0, 0.4)
                time.sleep(sleep)
                continue
            resp.raise_for_status()
            return resp
        except (requests.ReadTimeout, requests.ConnectTimeout, requests.ChunkedEncodingError) as e:
            last_err = e
            sleep = (backoff ** (attempt - 1)) + random.uniform(0, 0.4)
            time.sleep(sleep)
            continue
        except requests.RequestException as e:
            # Non-retryable (DNS, SSL, etc.) – bubble up after short delay
            last_err = e
            break

    # All attempts failed
    if last_err:
        raise last_err
    raise RuntimeError(f"Failed to GET {url} for unknown reasons.")


In [92]:
#two def functions to get the player-id, by either using the url or player name
SESSION = build_session()

def extract_player_id_from_url(url: str) -> str | None:
    m = re.search(r"/spieler/(\d+)", url)
    return m.group(1) if m else None

#NOTE -> by default it scrapps .de switch to to .com, .co.uk, .es etc.
def find_player_id_by_search(name: str, tld: str = "de", pause: float = 1.0) -> str | None:
    """Scrape Transfermarkt quick search results and return the first 'Spieler' hit ID."""
    url = f"https://www.transfermarkt.{tld}/schnellsuche/ergebnis/schnellsuche?query={requests.utils.quote(name)}"
    r = fetch(url, read_timeout=25.0)
    if r.status_code != 200:
        return None
    time.sleep(pause)  # be polite

    soup = BeautifulSoup(r.text, "html.parser")
    # Look for a link that contains '/spieler/<id>' (profil or main)
    for a in soup.select('a[href*="/spieler/"]'):
        m = re.search(r"/spieler/(\d+)", a.get("href", ""))
        if m:
            return m.group(1)
    return None

In [ ]:
pn_df=[]
name_player=list(df['Player'].unique())

for np in name_player:
    player_name=np
    
    pn_df.append(
        {
            'Player':player_name,
            'Player-ID':int(find_player_id_by_search(player_name)) if find_player_id_by_search(player_name) else 'No ID'
        }
    )

test=pd.DataFrame(pn_df)
test

,Player,Player-ID
0,Aaron Connolly,434207
1,Aaron Hickey,591949
2,Aaron Leya Iseka,289587
3,Aaron Mooy,123951
4,Aaron Ramsdale,427568
...,...,...
2029,Óliver Torres,214775
2030,Óscar Melendo,448344
2031,Óscar Mingueza,331505
2032,Ömer Toprak,43512


## Getting the Rumors of the Players

In this section we will get all the rumors associated with a player.

In [93]:
def _parse_rumours_page(html: str) -> pd.DataFrame:
    """
    Parse exactly the 'Gerüchtearchiv' table(s) from a rumours page (any page number).
    Returns a DataFrame with columns:
      ['club_name','club_id','last_source','last_answer','probability_raw']
    """
    soup = BeautifulSoup(html, "html.parser")

    def _parse_table(table) -> list[dict]:
        rows_out = []
        if not table:
            return rows_out
        for tr in table.select("tbody tr"):
            tds = tr.find_all("td")
            if len(tds) < 5:
                continue

            # interested club (2nd td)
            club_link = tds[1].select_one("a[href*='/verein/']")
            club_name = club_link.get_text(strip=True) if club_link else tds[1].get_text(strip=True)
            club_href = club_link.get("href", "") if club_link else ""
            m = re.search(r"/verein/(\d+)", club_href or "")
            club_id = m.group(1) if m else None

            # dates and probability
            last_source = tds[2].get_text(strip=True)
            last_answer = tds[3].get_text(strip=True)
            prob_text   = tds[4].get_text(strip=True).replace("–", "-")

            rows_out.append({
                "club_name": safe_strip(club_name),
                "club_id":   safe_strip(club_id),
                "last_source": safe_strip(last_source),
                "last_answer": safe_strip(last_answer),
                "probability_raw": safe_strip(prob_text),
            })
        return rows_out

    # 1) Prefer tables inside a box whose <h2> == "Gerüchtearchiv"
    all_rows = []
    for h2 in soup.find_all("h2"):
        if safe_strip(h2.get_text()).casefold() == "gerüchtearchiv":
            box = h2.find_parent(class_="box")
            if not box:
                continue
            tables = box.select("table.items")
            for tbl in tables:
                all_rows.extend(_parse_table(tbl))

    if all_rows:
        return pd.DataFrame(all_rows, columns=["club_name","club_id","last_source","last_answer","probability_raw"])

    # 2) Fallback: try pandas.read_html to find a table with the expected header
    try:
        dfs = pd.read_html(html, extract_links="body")
        for df in dfs:
            if "Interessierter Verein" in df.columns:
                # unpack tuple cells from extract_links="body"
                def unpack(cell):
                    if isinstance(cell, tuple):
                        return cell[0], cell[1]
                    return cell, None

                rows = []
                for _, row in df.iterrows():
                    club_text, club_href = unpack(row.get("Interessierter Verein"))
                    last_source, _ = unpack(row.get("Letzter Quelleneintrag"))
                    last_answer, _ = unpack(row.get("Letzte Antwort"))
                    prob_text, _  = unpack(row.get("Wechselwahrscheinlichkeit"))

                    club_id = None
                    if club_href:
                        m = re.search(r"/verein/(\d+)", club_href or "")
                        if m:
                            club_id = m.group(1)

                    rows.append({
                        "club_name": safe_strip(club_text),
                        "club_id":   safe_strip(club_id),
                        "last_source": safe_strip(last_source),
                        "last_answer": safe_strip(last_answer),
                        "probability_raw": safe_strip(prob_text),
                    })
                if rows:
                    return pd.DataFrame(rows, columns=["club_name","club_id","last_source","last_answer","probability_raw"])
    except ValueError:
        pass  # no tables found by read_html

    # 3) Final generic fallback: first table.items on the page
    generic_tbl = soup.select_one("table.items")
    if generic_tbl:
        rows = _parse_table(generic_tbl)
        return pd.DataFrame(rows, columns=["club_name","club_id","last_source","last_answer","probability_raw"])

    # Empty stable schema
    return pd.DataFrame(columns=["club_name","club_id","last_source","last_answer","probability_raw"])


def safe_strip(x):
    """
    Normalize a cell value to a clean string.
    - Unpacks (text, href) tuples from pandas.read_html(extract_links="body")
    - Handles None / NaN / floats
    """
    if isinstance(x, tuple):
        x = x[0]
    if x is None:
        return ""
    try:
        # handle pandas/numpy NaN without importing numpy explicitly
        if isinstance(x, float) and pd.isna(x):
            return ""
    except Exception:
        pass
    return str(x).strip()

def _to_dt(s) -> pd.Timestamp | None:
    """
    Parse Transfermarkt date strings (e.g., '11.06.2025') to Timestamp.
    Returns None if empty or '-' or unparsable.
    """
    s = safe_strip(s)
    if not s or s == "-":
        return None
    ts = pd.to_datetime(s, dayfirst=True, errors="coerce")
    return None if pd.isna(ts) else ts

def _parse_prob(x) -> float | None:
    """
    Parse probability strings like '75%' or '-' to a float (0..100) or None.
    """
    s = safe_strip(x).replace("%", "").replace(",", ".")
    if not s or s == "-":
        return None
    try:
        return float(s)
    except ValueError:
        return None

def _clean(s: str) -> str:
    """Collapse whitespace and trim."""
    return re.sub(r"\s+", " ", s).strip()

@lru_cache(maxsize=1024)
def get_club_name(club_id: str | int, tld: str = "de", pause: float = 0.5) -> str | None:
    """
    Resolve a club_id to its display name by scraping the club rumours page.
    Strategies: header <h1>, og:title, <title>, breadcrumb.
    Cached for speed/stability.
    """
    url = f"https://www.transfermarkt.{tld}/x/geruechte/verein/{club_id}"
    r = SESSION.get(url, timeout=20)
    if r.status_code != 200:
        return None
    time.sleep(pause)

    soup = BeautifulSoup(r.text, "html.parser")

    # 1) Main H1 headline
    h1 = soup.select_one(".data-header__headline-container h1")
    if h1 and _clean(h1.text):
        return _clean(h1.text)

    # 2) og:title (strip suffix like " – Gerüchte ...")
    og = soup.find("meta", attrs={"property": "og:title"})
    if og and og.get("content"):
        title = re.sub(r"\s*[-|–]\s*Gerüchte.*$", "", og["content"], flags=re.IGNORECASE)
        title = _clean(title)
        if title:
            return title

    # 3) <title>
    title_tag = soup.find("title")
    if title_tag and title_tag.text:
        title = re.sub(r"\s*[-|–]\s*Gerüchte.*$", "", title_tag.text, flags=re.IGNORECASE)
        title = _clean(title)
        if title:
            return title

    # 4) Breadcrumb fallback
    bc = soup.select_one(".tm-breadcrumb a[href*='/verein/']")
    if bc and _clean(bc.text):
        return _clean(bc.text)

    return None


def _find_archive_grid_info(html: str) -> tuple[str|None, str|None]:
    """
    Return (slug_path, grid_id) for the Gerüchtearchiv grid.
    slug_path example: '/nick-woltemade/geruechte/spieler/182913'
    grid_id example: 'yw2'
    """
    soup = BeautifulSoup(html, "html.parser")

    # slug from the grid's keys 'title' (most reliable)
    slug_path = None
    keys = soup.select_one("div.grid-view div.keys") or soup.select_one("div.keys")
    if keys and keys.has_attr("title"):
        slug_path = keys["title"].strip()

    # find the specific grid-view that contains the archive table
    grid = None
    for gv in soup.select("div.grid-view"):
        if gv.select_one("table.items thead th") and (gv.select_one("div.keys") or keys):
            # ensure this grid has the 'Interessierter Verein' header (archive table)
            head_txt = " ".join(th.get_text(" ", strip=True) for th in gv.select("thead th"))
            if "Interessierter Verein" in head_txt:
                grid = gv
                break
    grid_id = grid.get("id") if grid else None

    return slug_path, grid_id

def get_player_rumours_archive(
    player_id: str,
    tld: str = "de",
    pause: float = 0.7,
    fill_club_names: bool = True,
    max_pages: int | None = None,
    debug: bool = False,
) -> pd.DataFrame:
    base = f"https://www.transfermarkt.{tld}"

    def empty_df():
        df = pd.DataFrame(columns=["club_name","club_id","last_source","last_answer","probability_raw","page"])
        df["last_source_dt"] = pd.NaT
        df["last_answer_dt"] = pd.NaT
        df["probability_pct"] = pd.Series(dtype="float")
        return df

    # --- fetch page 1 (normal HTML) ---
    url_p1 = f"{base}/x/geruechte/spieler/{player_id}"
    r = SESSION.get(url_p1, timeout=20, allow_redirects=True)
    if r.status_code != 200:
        if debug: print(f"GET p1 failed: {r.status_code}")
        return empty_df()
    html1 = r.text
    time.sleep(pause)

    # parse page 1
    df_all = []
    df1 = _parse_rumours_page(html1)
    if not df1.empty:
        df1["page"] = 1
        df_all.append(df1)

    # discover slug and grid id for AJAX
    slug_path, grid_id = _find_archive_grid_info(html1)
    if debug: print("slug_path:", slug_path, "grid_id:", grid_id)

    # detect last page from the pager on page 1 (if present)
    def get_last_page(html: str) -> int:
        soup = BeautifulSoup(html, "html.parser")
        pager = soup.select_one("div.grid-view .tm-pagination") or soup.select_one(".tm-pagination")
        if not pager:
            return 1
        nums = []
        for a in pager.select("a.tm-pagination__link"):
            txt = (a.text or "").strip()
            if txt.isdigit():
                nums.append(int(txt))
            else:
                m = re.search(r"/page/(\d+)$", a.get("href") or "")
                if m:
                    nums.append(int(m.group(1)))
        return max(nums) if nums else 1

    last_page = get_last_page(html1)
    if max_pages is not None:
        last_page = min(last_page, max_pages)

    if debug: print("last_page:", last_page)

    # If no slug or grid id or only one page -> done
    if not slug_path or not grid_id or last_page <= 1:
        # finalize types/sort
        if not df_all:
            return empty_df()
        df = pd.concat(df_all, ignore_index=True)
        df["last_source_dt"] = df["last_source"].map(_to_dt)
        df["last_answer_dt"] = df["last_answer"].map(_to_dt)
        df["probability_pct"] = df["probability_raw"].map(_parse_prob)
        if fill_club_names:
            need = df["club_name"].astype(str).str.strip().eq("") & df["club_id"].astype(str).str.strip().ne("")
            ids = df.loc[need, "club_id"].astype(str).unique()
            if len(ids):
                mp = {cid: get_club_name(cid, tld=tld) for cid in ids}
                df.loc[need, "club_name"] = df.loc[need, "club_id"].astype(str).map(mp).fillna(df.loc[need, "club_name"])
        return df.sort_values(["last_answer_dt","last_source_dt","page"], ascending=[False,False,True], na_position="last").reset_index(drop=True)

    # --- fetch pages 2..last via AJAX endpoint ---
    # Pattern: /<slug>/ajax/<grid_id>/page/<n>
    for n in range(2, last_page + 1):
        ajax_url = urljoin(base, f"{slug_path}/ajax/{grid_id}/page/{n}")
        if debug: print("GET", ajax_url)
        r = SESSION.get(ajax_url, timeout=20, headers={"Referer": urljoin(base, slug_path)}, allow_redirects=True)
        if r.status_code != 200:
            if debug: print(" -> status", r.status_code)
            break
        html = r.text
        # the AJAX response is a fragment that still contains the table; parse it
        dfp = _parse_rumours_page(html)
        if dfp.empty:
            if debug: print(" -> empty page, stopping")
            break
        dfp["page"] = n
        df_all.append(dfp)
        time.sleep(pause)

    # finalize
    if not df_all:
        return empty_df()
    df = pd.concat(df_all, ignore_index=True)
    df["last_source_dt"] = df["last_source"].map(_to_dt)
    df["last_answer_dt"] = df["last_answer"].map(_to_dt)
    df["probability_pct"] = df["probability_raw"].map(_parse_prob)
    if fill_club_names:
        need = df["club_name"].astype(str).str.strip().eq("") & df["club_id"].astype(str).str.strip().ne("")
        ids = df.loc[need, "club_id"].astype(str).unique()
        if len(ids):
            mp = {cid: get_club_name(cid, tld=tld) for cid in ids}
            df.loc[need, "club_name"] = df.loc[need, "club_id"].astype(str).map(mp).fillna(df.loc[need, "club_name"])
    return df.sort_values(["last_answer_dt","last_source_dt","page"], ascending=[False,False,True], na_position="last").reset_index(drop=True)

In [85]:
archive = get_player_rumours_archive("182913", tld="de", debug=True)
archive['Player']=test[test['Player-ID'].eq(182913)]['Player'].iloc[0]
archive

slug_path: /x/geruechte/spieler/182913 grid_id: yw1
last_page: 3
GET https://www.transfermarkt.de/x/geruechte/spieler/182913/ajax/yw1/page/2
GET https://www.transfermarkt.de/x/geruechte/spieler/182913/ajax/yw1/page/3


,club_name,club_id,last_source,last_answer,probability_raw,page,last_source_dt,last_answer_dt,probability_pct,Player
0,AC Mailand,5,11.06.2025,11.06.2025,-,1,2025-06-11,2025-06-11,None,Adrien Rabiot
1,Olympique Marseille,244,15.09.2024,15.09.2024,-,1,2024-09-15,2024-09-15,None,Adrien Rabiot
2,Newcastle United,762,13.09.2024,13.09.2024,-,1,2024-09-13,2024-09-13,None,Adrien Rabiot
3,Manchester United,985,10.09.2024,11.09.2024,-,1,2024-09-10,2024-09-11,None,Adrien Rabiot
4,Galatasaray,141,08.09.2024,08.09.2024,-,1,2024-09-08,2024-09-08,None,Adrien Rabiot
5,Al-Nassr FC,18544,08.09.2024,08.09.2024,-,1,2024-09-08,2024-09-08,None,Adrien Rabiot
6,FC Bayern München,27,28.08.2024,29.08.2024,-,1,2024-08-28,2024-08-29,None,Adrien Rabiot
7,FC Liverpool,31,22.08.2024,23.08.2024,-,1,2024-08-22,2024-08-23,None,Adrien Rabiot
8,Real Madrid,418,08.07.2024,18.07.2024,-,1,2024-07-08,2024-07-18,None,Adrien Rabiot
9,Manchester United,985,02.05.2024,02.05.2024,-,1,2024-05-02,2024-05-02,None,Adrien Rabiot


## Full Run

In [94]:
pn_df=[]
name_player=list(df['Player'].unique())

for np in name_player:
    player_name=np
    player_id_=find_player_id_by_search(player_name)

    pn_df.append(
        {
            'Player':player_name,
            'Player-ID':int(player_id_) if player_id_ else 'No ID'
        }
    )

name_id=pd.DataFrame(pn_df)

rumor_df=pd.DataFrame()
for id in list(name_id['Player-ID']):
    archive_rumor = get_player_rumours_archive(id, tld="de", debug=True)
    archive_rumor['Player']=name_id[name_id['Player-ID'].eq(id)]['Player'].iloc[0]
    rumor_df=pd.concat([rumor_df,archive_rumor],axis=0)
rumor_df

slug_path: /x/geruechte/spieler/434207 grid_id: yw2
last_page: 1
slug_path: /x/geruechte/spieler/591949 grid_id: yw2
last_page: 1
slug_path: /x/geruechte/spieler/289587 grid_id: yw2
last_page: 1
slug_path: /x/geruechte/spieler/123951 grid_id: yw2
last_page: 1
slug_path: /x/geruechte/spieler/427568 grid_id: yw2
last_page: 1
slug_path: /x/geruechte/spieler/50057 grid_id: yw2
last_page: 2
GET https://www.transfermarkt.de/x/geruechte/spieler/50057/ajax/yw2/page/2
slug_path: /x/geruechte/spieler/477758 grid_id: yw2
last_page: 1
slug_path: /x/geruechte/spieler/251878 grid_id: yw2
last_page: 2
GET https://www.transfermarkt.de/x/geruechte/spieler/251878/ajax/yw2/page/2
slug_path: /x/geruechte/spieler/776798 grid_id: yw2
last_page: 1


C:\Users\gandj\AppData\Local\Temp\ipykernel_12836\2501677276.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rumor_df=pd.concat([rumor_df,archive_rumor],axis=0)


slug_path: /x/geruechte/spieler/724520 grid_id: yw2
last_page: 1


C:\Users\gandj\AppData\Local\Temp\ipykernel_12836\2501677276.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rumor_df=pd.concat([rumor_df,archive_rumor],axis=0)


slug_path: /x/geruechte/spieler/340394 grid_id: yw2
last_page: 1


...

C:\Users\gandj\AppData\Local\Temp\ipykernel_12836\2501677276.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rumor_df=pd.concat([rumor_df,archive_rumor],axis=0)


slug_path: /x/geruechte/spieler/43512 grid_id: yw2
last_page: 2
GET https://www.transfermarkt.de/x/geruechte/spieler/43512/ajax/yw2/page/2


C:\Users\gandj\AppData\Local\Temp\ipykernel_12836\2501677276.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rumor_df=pd.concat([rumor_df,archive_rumor],axis=0)


slug_path: /x/geruechte/spieler/53622 grid_id: yw2
last_page: 3
GET https://www.transfermarkt.de/x/geruechte/spieler/53622/ajax/yw2/page/2
GET https://www.transfermarkt.de/x/geruechte/spieler/53622/ajax/yw2/page/3


,club_name,club_id,last_source,last_answer,probability_raw,page,last_source_dt,last_answer_dt,probability_pct,Player
0,Celtic Glasgow,371,19.08.2024,19.08.2024,-,1,2024-08-19,2024-08-19,NaN,Aaron Connolly
1,FC Middlesbrough,641,29.12.2021,29.12.2021,-,1,2021-12-29,2021-12-29,NaN,Aaron Connolly
2,TSG 1899 Hoffenheim,533,22.11.2018,22.11.2018,-,1,2018-11-22,2018-11-22,NaN,Aaron Connolly
0,Manchester United,985,25.09.2023,25.09.2023,-,1,2023-09-25,2023-09-25,NaN,Aaron Hickey
1,FC Arsenal,11,25.09.2023,25.09.2023,-,1,2023-09-25,2023-09-25,NaN,Aaron Hickey
...,...,...,...,...,...,...,...,...,...,...
30,Hamburger SV,41,03.05.2011,05.05.2011,-,2,2011-05-03,2011-05-05,NaN,İlkay Gündoğan
31,Olympique Lyon,1041,05.04.2011,05.04.2011,-,3,2011-04-05,2011-04-05,NaN,İlkay Gündoğan
32,SV Werder Bremen,86,11.01.2011,13.01.2011,-,3,2011-01-11,2011-01-13,NaN,İlkay Gündoğan
33,TSG 1899 Hoffenheim,533,11.08.2010,29.08.2010,-,3,2010-08-11,2010-08-29,NaN,İlkay Gündoğan


In [99]:
rumor_df.to_csv('..\..\DataSources\Processed\Rumor_Overview.csv',index=False)

## Creating the final Rumor Data Frame

In [5]:
club_league_df=df[['Club_Joined','League_Joined']].drop_duplicates()
club_league_df.rename(columns={'Club_Joined':'club_name'},inplace=True)
club_league_df.head()

,club_name,League_Joined
0,Venezia FC,Other Leagues
1,Hull City,Other Leagues
2,FC Brentford,Premier League
3,FC Metz,Ligue 1
4,Shanghai Port,Other Leagues


In [7]:
df_rumor=pd.read_csv(r'..\..\DataSources\Processed\Rumor_Overview.csv').drop(['probability_raw','page','probability_pct','club_id'],axis=1)
df_rumor['last_source_dt'] = pd.to_datetime(df_rumor['last_source_dt'], errors='coerce')
df_rumor['Performance_Year']=df_rumor['last_source_dt'].dt.year
#df_rumor['Performance_Month']=df_rumor['last_source_dt'].dt.month
df_rumor.drop(['last_source','last_answer','last_source_dt',],axis=1,inplace=True)

df_final=pd.merge(df_rumor,club_league_df,on='club_name',how='left')
df_final

,club_name,last_answer_dt,Player,Performance_Year,League_Joined
0,Celtic Glasgow,2024-08-19,Aaron Connolly,2024.0,Other Leagues
1,FC Middlesbrough,2021-12-29,Aaron Connolly,2021.0,Other Leagues
2,TSG 1899 Hoffenheim,2018-11-22,Aaron Connolly,2018.0,Bundesliga
3,Manchester United,2023-09-25,Aaron Hickey,2023.0,Premier League
4,FC Arsenal,2023-09-25,Aaron Hickey,2023.0,Premier League
...,...,...,...,...,...
30170,Olympique Lyon,2011-04-05,İlkay Gündoğan,2011.0,Ligue 1
30171,SV Werder Bremen,2011-01-13,İlkay Gündoğan,2011.0,Bundesliga
30172,SV Werder Bremen,2011-01-13,İlkay Gündoğan,2011.0,Other Leagues
30173,TSG 1899 Hoffenheim,2010-08-29,İlkay Gündoğan,2010.0,Bundesliga


### Count of Leagues by Player

We use the dataframe from the previous section, where all stats and moves of players are included. Next, we will combine it with the rumors table, but first we need to aggregate the league apprance by player, league and year.

In [ ]:
df_rumor_agg=df_final.groupby(
    [
        'Player', 'Performance_Year', 'League_Joined'
    ]
    ).size().unstack(fill_value=0).rename(columns=lambda c: f'{c}_Rumors').reset_index().rename_axis(columns=' ')
df_rumor_agg

,Player,Performance_Year,Bundesliga_Rumors,LaLiga_Rumors,Ligue 1_Rumors,Other Leagues_Rumors,Premier League_Rumors,Serie A_Rumors
0,Aaron Connolly,2018.0,1,0,0,0,0,0
1,Aaron Connolly,2021.0,0,0,0,1,0,0
2,Aaron Connolly,2024.0,0,0,0,1,0,0
3,Aaron Hickey,2020.0,1,0,0,0,0,1
4,Aaron Hickey,2022.0,0,0,0,2,4,2
...,...,...,...,...,...,...,...,...
9408,İlkay Gündoğan,2018.0,0,0,0,0,0,1
9409,İlkay Gündoğan,2022.0,1,1,0,1,0,1
9410,İlkay Gündoğan,2023.0,1,1,0,0,1,0
9411,İlkay Gündoğan,2024.0,0,0,0,2,1,0


In [30]:
df_final=pd.merge(df,df_rumor_agg,on=['Player','Performance_Year'],how='left').fillna(0)
df_final.to_csv('..\..\DataSources\Processed\Cleaned_Final_Stats_w_Rumors.csv',index=False)